In [1]:
import tensorflow as tf
import numpy as np
import cv2 as cv


In [57]:
def parse_image(filename):
  parts = tf.strings.split(filename, os.sep)
  label = parts[-2]

  image = tf.io.read_file(filename)
  image = tf.io.decode_png(image)
  image = tf.image.convert_image_dtype(image, tf.float32)
  image = tf.image.resize(image, [64, 64])
  return image, label

def threshold_image(image):
  image  = (image.numpy()*255).astype(np.uint8)
  image = cv.adaptiveThreshold(image,255,cv.ADAPTIVE_THRESH_GAUSSIAN_C,\
            cv.THRESH_BINARY,75,10)
  image = image/255
  return image.reshape((64,64,1))

def tf_threshold_image(image, label):
  im_shape = image.shape
  [image,] = tf.py_function(threshold_image, [image], [tf.float32])
  image.set_shape(im_shape)
  return image, label

def _map_func(image, label):
  x = str(np.array(label)).split('\'')
  label = int(x[1])
  label = tf.one_hot(label, 10, name='label', axis=-1)

  return image, label

In [58]:
'''
Function takes file name and generates appropriate tf.Dataset
'''
def getDataset(filePath):
    list_ds = tf.data.Dataset.list_files(str(filePath) + ('/*/*'))
    images_ds = list_ds.map(parse_image)
    threshed_ds = images_ds.map(tf_threshold_image)
    threshed_ds = threshed_ds.map(_map_func)
    return threshed_ds


In [59]:
train = getDataset('Dataset/training_data')
test = getDataset('Dataset/testing_data')

NotImplementedError: in user code:

    File "<ipython-input-57-531c29286aef>", line 25, in _map_func  *
        x = str(np.array(label)).split('\'')

    NotImplementedError: Cannot convert a symbolic Tensor (args_1:0) to a numpy array. This error may indicate that you're trying to pass a Tensor to a NumPy call, which is not supported


In [42]:
#making model
from tensorflow.keras import layers, models

rescale = tf.keras.Sequential([
  layers.Rescaling(1./255)
])
'''
model = models.Sequential()
model.add(rescale)
model.add(layers.Conv2D(32, (5, 5), activation='relu', input_shape=(64, 64, 1)))
model.add(layers.Conv2D(filters = 32, kernel_size = 5, strides = 1, use_bias=False))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10))
'''
model = tf.keras.Sequential([
  layers.Conv2D(32, (5, 5), activation='relu', input_shape=(64, 64, 1)),
  layers.MaxPooling2D(2),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(2),
  layers.Flatten(),
  layers.Dense(64, activation='relu'),
  layers.Dense(10,  activation="softmax", name="predictions"),
])


In [43]:
model.summary()

Model: "sequential_26"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_29 (Conv2D)          (None, 60, 60, 32)        832       
                                                                 
 max_pooling2d_21 (MaxPoolin  (None, 30, 30, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_30 (Conv2D)          (None, 30, 30, 16)        4624      
                                                                 
 max_pooling2d_22 (MaxPoolin  (None, 15, 15, 16)       0         
 g2D)                                                            
                                                                 
 flatten_12 (Flatten)        (None, 3600)              0         
                                                                 
 dense_23 (Dense)            (None, 64)              

In [46]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

In [47]:
model.fit(train, epochs=2)

Epoch 1/2


ValueError: in user code:

    File "/home/alex/greycroc/lib/python3.8/site-packages/keras/engine/training.py", line 1021, in train_function  *
        return step_function(self, iterator)
    File "/home/alex/greycroc/lib/python3.8/site-packages/keras/engine/training.py", line 1010, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/alex/greycroc/lib/python3.8/site-packages/keras/engine/training.py", line 1000, in run_step  **
        outputs = model.train_step(data)
    File "/home/alex/greycroc/lib/python3.8/site-packages/keras/engine/training.py", line 860, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/home/alex/greycroc/lib/python3.8/site-packages/keras/engine/training.py", line 918, in compute_loss
        return self.compiled_loss(
    File "/home/alex/greycroc/lib/python3.8/site-packages/keras/engine/compile_utils.py", line 201, in __call__
        loss_value = loss_obj(y_t, y_p, 